In [14]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()

In [ ]:
connection.close()

In [6]:
request_id = -999

In [7]:
vm_query = f'''WITH ClonedRecords AS (
	SELECT 
        LPAD(stg.bic_vendor_facility::VARCHAR, 3, '0') || LPAD(stg.bic_vendor_number::VARCHAR, 6, '0') AS home_destination_insert, -- Concatenate and pad
		af.facility_code::numeric,stg.*   
		FROM XXPO_ORCL2CSPOMS_VEND_STG stg
		CROSS JOIN (
		    SELECT facility_code
		    FROM cspoms_facility_master
		    WHERE inactive_date IS NULL
		) af
		WHERE stg.process_status IN ('N')
		and Request_id = '593'
		  AND (
		    stg.vendor_foodlink_flg IN ('1', '2') 
		    OR stg.vendor_freight_flg = 'Y'
		    OR stg.bic_vendor_facility IN (14, 21)
		  )
		  AND stg.bic_vendor_facility != af.facility_code::numeric
)
INSERT INTO cspoms.XXPO_VENDOR_MASTER (
    CS_VENDOR_ID,
    home_destination,
    VENDOR_NUMBER,
    VENDOR_FACILITY,
    CS_VEND_SITE_ID,
    CONNECTING_VENDOR,
    VENDOR_NAME,
    VENDOR_TYPE,
    FACILITY_NAME,
    VENDOR_STATUS,
    AP_VENDOR_NBR,
    AP_VENDOR_NAME,
    BUYER_NUMBER,
    BUYER_ID,
    BUYER_NAME,
    LOAD_BUILDING,
    WAREHOUSE_CODE,
    FOOD_LINK_FLAG,
    ADDR_LINE1,
    ADDR_LINE2,
    ADDR_LINE3,
    CITY,
    STATE,
    ZIP_CODE,
    COUNTRY,
    PHONE_NBR,
    VENDOR_CONTACT_NAME,
    SHIP_TO_LOCATION_NAME,
    BILL_TO_LOCATION_NAME,
    TRUCK_FLAG,
    RAIL_FLAG,
    PO_TRANSMISSION_TYPE,
    PO_TRANSMISSION_VALUE,
    BROKER_NUMBER,
    PICK_MON,
    PICK_TUE,
    PICK_WED,
    PICK_THR,
    PICK_FRI,
    PICK_SAT,
    PICK_SUN,
    AFE_CONTACT,
    AFE_FAX_NUMBER,
    AFE_TYPE,
    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT 
    nextval('cs_vendor_id_seq'),
    home_destination_insert, 
    bic_vendor_number,
    lpad(facility_code::varchar,3,'0'),
    vendor_site_id,
    connecting_vendor,
    bic_name,
    'C',
    ship_to_location_name,
    bic_status,
    bic_payable_vendor_nbr,
    ap_vendor_name,
    vendor_buyer_nbr,
    bic_buyer_number,
    buyer_name,
    load_building,
    bic_warehouse_code,
    vendor_foodlink_flg,
    bic_address_1,
    addr_line2,
    addr_line3,
    bic_city,
    bic_state,
    bic_zip,
    bic_country,
    bic_phone,
    bic_contact,
    ship_to_location_name,
    bill_to_location_name,
    bic_flag_truck,
    bic_flag_rail,
    bic_afe_contact, 
    bic_afe_fax_number, 
    bic_broker_number,
    pickup_day_1,
    pickup_day_2,
    pickup_day_3,
    pickup_day_4,
    pickup_day_5,
    pickup_day_6,
    pickup_day_7,
    bic_afe_contact,
    bic_afe_fax_number,
    bic_afe_type1,
    {request_id}, 
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM ClonedRecords
ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET
    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
    VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
    CS_VEND_SITE_ID = EXCLUDED.CS_VEND_SITE_ID,
    CONNECTING_VENDOR = EXCLUDED.CONNECTING_VENDOR,
    VENDOR_NAME = EXCLUDED.VENDOR_NAME,
    VENDOR_TYPE = EXCLUDED.VENDOR_TYPE,
    FACILITY_NAME = EXCLUDED.FACILITY_NAME,
    VENDOR_STATUS = EXCLUDED.VENDOR_STATUS,
    AP_VENDOR_NBR = EXCLUDED.AP_VENDOR_NBR,
    AP_VENDOR_NAME = EXCLUDED.AP_VENDOR_NAME,
    BUYER_NUMBER = EXCLUDED.BUYER_NUMBER,
    BUYER_ID = EXCLUDED.BUYER_ID,
    BUYER_NAME = EXCLUDED.BUYER_NAME,
    LOAD_BUILDING = EXCLUDED.LOAD_BUILDING,
    WAREHOUSE_CODE = EXCLUDED.WAREHOUSE_CODE,
    FOOD_LINK_FLAG = EXCLUDED.FOOD_LINK_FLAG,
    ADDR_LINE1 = EXCLUDED.ADDR_LINE1,
    ADDR_LINE2 = EXCLUDED.ADDR_LINE2,
    ADDR_LINE3 = EXCLUDED.ADDR_LINE3,
    CITY = EXCLUDED.CITY,
    STATE = EXCLUDED.STATE,
    ZIP_CODE = EXCLUDED.ZIP_CODE,
    COUNTRY = EXCLUDED.COUNTRY,
    PHONE_NBR = EXCLUDED.PHONE_NBR,
    VENDOR_CONTACT_NAME = EXCLUDED.VENDOR_CONTACT_NAME,
    SHIP_TO_LOCATION_NAME = EXCLUDED.SHIP_TO_LOCATION_NAME,
    BILL_TO_LOCATION_NAME = EXCLUDED.BILL_TO_LOCATION_NAME,
    TRUCK_FLAG = EXCLUDED.TRUCK_FLAG,
    RAIL_FLAG = EXCLUDED.RAIL_FLAG,
    PO_TRANSMISSION_TYPE = EXCLUDED.PO_TRANSMISSION_TYPE,
    PO_TRANSMISSION_VALUE = EXCLUDED.PO_TRANSMISSION_VALUE,
    BROKER_NUMBER = EXCLUDED.BROKER_NUMBER,
    PICK_MON = EXCLUDED.PICK_MON,
    PICK_TUE = EXCLUDED.PICK_TUE,
    PICK_WED = EXCLUDED.PICK_WED,
    PICK_THR = EXCLUDED.PICK_THR,
    PICK_FRI = EXCLUDED.PICK_FRI,
    PICK_SAT = EXCLUDED.PICK_SAT,
    PICK_SUN = EXCLUDED.PICK_SUN,
    AFE_CONTACT = EXCLUDED.AFE_CONTACT,
    AFE_FAX_NUMBER = EXCLUDED.AFE_FAX_NUMBER,
    AFE_TYPE = EXCLUDED.AFE_TYPE,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
'''
post_cur.execute(vm_query)
vm_count = post_cur.rowcount
vm_count

799204

In [11]:
vop_query = f'''
WITH CrossJoinTemp AS (
--    select count(*)
--    from
--    (    000118
SELECT 
        LPAD(stg.bic_vendor_facility::VARCHAR, 3, '0') || LPAD(stg.bic_vendor_number::VARCHAR, 6, '0') AS home_destination_insert,
        af.facility_code::numeric AS facility_code,
        stg.*  
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric
--    		  ) as subquery

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, VendorOrderPoint AS (
    SELECT 
        vm.cs_vendor_id,
        temp.*
    FROM CrossJoinTemp temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

INSERT INTO XXPO_VENDOR_ORDER_POINT (
    CS_VENDOR_ID,
    MAIN_FACILITY,
    BUILD_TO_DAYS_SUPPLY_FLAG,
    DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS,

    LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7,

    APPLY_RATIONING,
    DEMAND_DAYS,
    LAG_DAYS,
    OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7,

    OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD,
    PULLBACK,

    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    CS_VENDOR_ID,
    MAIN_FACILITY,
    build_to_days_supply,
    DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS,

    LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7,

    APPLY_RATIONING,
    DEMAND_DAYS,
    LAG_DAYS,
    OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7,

    OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD,
    pullbks_available,

	{request_id}, 
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM VendorOrderPoint
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    MAIN_FACILITY = EXCLUDED.MAIN_FACILITY,
    BUILD_TO_DAYS_SUPPLY_FLAG = EXCLUDED.BUILD_TO_DAYS_SUPPLY_FLAG,
    DAYS_TO_BUILD_TO = EXCLUDED.DAYS_TO_BUILD_TO,
    USE_CYCLE_STOCK_AS_MIN = EXCLUDED.USE_CYCLE_STOCK_AS_MIN,
    USE_FAC_WHSE_SS = EXCLUDED.USE_FAC_WHSE_SS,
    LEAD_TIME_OVRD_DAYS_1 = EXCLUDED.LEAD_TIME_OVRD_DAYS_1,
    LEAD_TIME_OVRD_DAYS_2 = EXCLUDED.LEAD_TIME_OVRD_DAYS_2,
    LEAD_TIME_OVRD_DAYS_3 = EXCLUDED.LEAD_TIME_OVRD_DAYS_3,
    LEAD_TIME_OVRD_DAYS_4 = EXCLUDED.LEAD_TIME_OVRD_DAYS_4,
    LEAD_TIME_OVRD_DAYS_5 = EXCLUDED.LEAD_TIME_OVRD_DAYS_5,
    LEAD_TIME_OVRD_DAYS_6 = EXCLUDED.LEAD_TIME_OVRD_DAYS_6,
    LEAD_TIME_OVRD_DAYS_7 = EXCLUDED.LEAD_TIME_OVRD_DAYS_7,
    APPLY_RATIONING = EXCLUDED.APPLY_RATIONING,
    DEMAND_DAYS = EXCLUDED.DEMAND_DAYS,
    LAG_DAYS = EXCLUDED.LAG_DAYS,
    OP_DAYS_1ST_RUN_1 = EXCLUDED.OP_DAYS_1ST_RUN_1,
    OP_DAYS_1ST_RUN_2 = EXCLUDED.OP_DAYS_1ST_RUN_2,
    OP_DAYS_1ST_RUN_3 = EXCLUDED.OP_DAYS_1ST_RUN_3,
    OP_DAYS_1ST_RUN_4 = EXCLUDED.OP_DAYS_1ST_RUN_4,
    OP_DAYS_1ST_RUN_5 = EXCLUDED.OP_DAYS_1ST_RUN_5,
    OP_DAYS_1ST_RUN_6 = EXCLUDED.OP_DAYS_1ST_RUN_6,
    OP_DAYS_1ST_RUN_7 = EXCLUDED.OP_DAYS_1ST_RUN_7,
    OP_DAYS_2ND_RUN_1 = EXCLUDED.OP_DAYS_2ND_RUN_1,
    OP_DAYS_2ND_RUN_2 = EXCLUDED.OP_DAYS_2ND_RUN_2,
    OP_DAYS_2ND_RUN_3 = EXCLUDED.OP_DAYS_2ND_RUN_3,
    OP_DAYS_2ND_RUN_4 = EXCLUDED.OP_DAYS_2ND_RUN_4,
    OP_DAYS_2ND_RUN_5 = EXCLUDED.OP_DAYS_2ND_RUN_5,
    OP_DAYS_2ND_RUN_6 = EXCLUDED.OP_DAYS_2ND_RUN_6,
    OP_DAYS_2ND_RUN_7 = EXCLUDED.OP_DAYS_2ND_RUN_7,
    SHIP_WITH_VENDOR = EXCLUDED.SHIP_WITH_VENDOR,
    HAWAII_TRANS_METHOD = EXCLUDED.HAWAII_TRANS_METHOD,
    PULLBACK = EXCLUDED.PULLBACK,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    CREATION_DATE = EXCLUDED.CREATION_DATE,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    CREATED_BY = EXCLUDED.CREATED_BY,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY'''

post_cur.execute(vop_query)
vop_count = post_cur.rowcount
vop_count

799204

In [12]:
voi_query = f'''
WITH CloneRecords AS (
SELECT 
        LPAD(stg.bic_vendor_facility::VARCHAR, 3, '0') || LPAD(stg.bic_vendor_number::VARCHAR, 6, '0') AS home_destination_insert,
        af.facility_code::numeric AS facility_code,
        stg.*  
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, VendorOrderInfo AS (
    SELECT 
        vm.cs_vendor_id,
        temp.*
    FROM CloneRecords temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

INSERT INTO XXPO_VENDOR_ORDER_INFO (
    CS_VENDOR_ID,
    TRANSIT_DAYS,
    LAG_DAYS,
    LEAD_TIME_STATED_TYPE,
    LEAD_TIME_STATED_DAYS,
    RECOMMENDED_FLAG,

    MINIMUM_BKT_0,
    MINIMUM_BKT_0_TYPE,

    MAXIMUM_BKT,
    MAXIMUM_BKT_TYPE,

    MIN_BKT_2ND,
    MIN_BKT_TYPE_2ND,

    MAX_BKT_2ND,
    MAX_BKT_TYPE_2ND,

    REMARK1_FLAG,
    REMARK1,
    REMARK2_FLAG,
    REMARK2,
    REMARK3_FLAG,
    REMARK3,
    REMARK4_FLAG,
    REMARK4,

    FIXED_REVIEW_MON,
    FIXED_REVIEW_TUE,
    FIXED_REVIEW_WED,
    FIXED_REVIEW_THU,
    FIXED_REVIEW_FRI,
    FIXED_REVIEW_SAT,
    FIXED_REVIEW_SUN,

    PICK_MON,
    PICK_TUE,
    PICK_WED,
    PICK_THR,
    PICK_FRI,
    PICK_SAT,
    PICK_SUN,

    VENDOR_SERVICE_LEVEL,
    DAYS_TO_BUY_OVERRIDE,
    ORDER_SEQUENCE,
    LOAD_BLDG_LIMIT_FLG,
    ORDER_FILTER_FLAG,

    SHORTED_DEALS_FLAG,
    EOQ_MIN,
    EOQ_MAX,
    EOQ_TYPE,

    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    CS_VENDOR_ID,
    transit_days::numeric,
    lag_days,
    lead_time_calendar_ind,
    vendor_lead_days::numeric,
    bic_po_recommended_ind,

    bic_minimum_quantity,
    bic_minimum_type,

    bic_maximum_quantity,
    bic_maximum_type,

    bic_2nd_min_quantity,
    bic_2nd_min_type,

    bic_2nd_max_quantity,
    bic_2nd_max_type,

    bic_fixed_review_day1,
    bic_fixed_review_day2,
    bic_fixed_review_day3,
    bic_fixed_review_day4,
    bic_fixed_review_day5,
    bic_fixed_review_day6,
    bic_fixed_review_day7,

    remark1_flag,
    remark1,
    remark2_flag,
    remark2,
    remark3_flag,
    remark3,
    remark4_flag,
    remark4,

    pickup_day_1,
    pickup_day_2,
    pickup_day_3,
    pickup_day_4,
    pickup_day_5,
    pickup_day_6,
    pickup_day_7,

    bic_target_service_level,
    days_to_buy_override,
    bic_order_sequence,
    load_building,
    bic_filter_vendor_flag,

    bic_bill_for_short_deals_flag,
    bic_current_bkt_number,
    bic_bracket_quantity,
    bic_current_bkt_type,

    {request_id},
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM VendorOrderInfo
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    TRANSIT_DAYS = EXCLUDED.TRANSIT_DAYS,
    LAG_DAYS = EXCLUDED.LAG_DAYS,
    LEAD_TIME_STATED_TYPE = EXCLUDED.LEAD_TIME_STATED_TYPE,
    LEAD_TIME_STATED_DAYS = EXCLUDED.LEAD_TIME_STATED_DAYS,
    RECOMMENDED_FLAG = EXCLUDED.RECOMMENDED_FLAG,
    MINIMUM_BKT_0 = EXCLUDED.MINIMUM_BKT_0,
    MINIMUM_BKT_0_TYPE = EXCLUDED.MINIMUM_BKT_0_TYPE,
    MAXIMUM_BKT = EXCLUDED.MAXIMUM_BKT,
    MAXIMUM_BKT_TYPE = EXCLUDED.MAXIMUM_BKT_TYPE,
    MIN_BKT_2ND = EXCLUDED.MIN_BKT_2ND,
    MIN_BKT_TYPE_2ND = EXCLUDED.MIN_BKT_TYPE_2ND,
    MAX_BKT_2ND = EXCLUDED.MAX_BKT_2ND,
    MAX_BKT_TYPE_2ND = EXCLUDED.MAX_BKT_TYPE_2ND,
    REMARK1_FLAG = EXCLUDED.REMARK1_FLAG,
    REMARK1 = EXCLUDED.REMARK1,
    REMARK2_FLAG = EXCLUDED.REMARK2_FLAG,
    REMARK2 = EXCLUDED.REMARK2,
    REMARK3_FLAG = EXCLUDED.REMARK3_FLAG,
    REMARK3 = EXCLUDED.REMARK3,
    REMARK4_FLAG = EXCLUDED.REMARK4_FLAG,
    REMARK4 = EXCLUDED.REMARK4,
    FIXED_REVIEW_MON = EXCLUDED.FIXED_REVIEW_MON,
    FIXED_REVIEW_TUE = EXCLUDED.FIXED_REVIEW_TUE,
    FIXED_REVIEW_WED = EXCLUDED.FIXED_REVIEW_WED,
    FIXED_REVIEW_THU = EXCLUDED.FIXED_REVIEW_THU,
    FIXED_REVIEW_FRI = EXCLUDED.FIXED_REVIEW_FRI,
    FIXED_REVIEW_SAT = EXCLUDED.FIXED_REVIEW_SAT,
    FIXED_REVIEW_SUN = EXCLUDED.FIXED_REVIEW_SUN,
    PICK_MON = EXCLUDED.PICK_MON,
    PICK_TUE = EXCLUDED.PICK_TUE,
    PICK_WED = EXCLUDED.PICK_WED,
    PICK_THR = EXCLUDED.PICK_THR,
    PICK_FRI = EXCLUDED.PICK_FRI,
    PICK_SAT = EXCLUDED.PICK_SAT,
    PICK_SUN = EXCLUDED.PICK_SUN,
    VENDOR_SERVICE_LEVEL = EXCLUDED.VENDOR_SERVICE_LEVEL,
    DAYS_TO_BUY_OVERRIDE = EXCLUDED.DAYS_TO_BUY_OVERRIDE,
    ORDER_SEQUENCE = EXCLUDED.ORDER_SEQUENCE,
    LOAD_BLDG_LIMIT_FLG = EXCLUDED.LOAD_BLDG_LIMIT_FLG,
    ORDER_FILTER_FLAG = EXCLUDED.ORDER_FILTER_FLAG,
    SHORTED_DEALS_FLAG = EXCLUDED.SHORTED_DEALS_FLAG,
    EOQ_MIN = EXCLUDED.EOQ_MIN,
    EOQ_MAX = EXCLUDED.EOQ_MAX,
    EOQ_TYPE = EXCLUDED.EOQ_TYPE,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
'''
post_cur.execute(voi_query)
voi_count = post_cur.rowcount
voi_count

799204

In [15]:
vci_query = f'''
WITH CloneRecords AS (
SELECT 
        LPAD(stg.bic_vendor_facility::VARCHAR, 3, '0') || LPAD(stg.bic_vendor_number::VARCHAR, 6, '0') AS home_destination_insert,
        af.facility_code::numeric AS facility_code,
        stg.*  
        
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, VendorCostingInfo AS (
    SELECT 
        vm.cs_vendor_id,
		temp.*
    FROM CloneRecords temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

-- Insert or upsert into XXPO_VENDOR_EDI_INFO
INSERT INTO XXPO_VENDOR_COSTING_INFO (
    CS_VENDOR_ID,
    LOAD_BUILDING,
    FREIGHT_ALLOW_BUY_RATE,
    FREIGHT_ALLOW_BUY_RATE_TYPE,
    FREIGHT_ALLOW_BUY_RATE_CHANGED,
    FREIGHT_BILL_BUY_RATE,
    FREIGHT_BILL_BUY_RATE_TYPE,
    FREIGHT_BILL_BUY_RATE_CHANGED,
    BACKHAUL_BUY_RATE,
    BACKHAUL_BUY_RATE_TYPE,
    BACKHAUL_BUY_RATE_CHANGED,
    PRE_PAY_FLAG,
    PRE_PAY_ADD_FLAG,
    FREIGHT_BILL_FLAG,
    BACKHAUL_FLAG,
    EXTERNAL_BACKHAUL_FLAG,
    SSA_RATE,
    SSA_RATE_TYPE,
    SSA_RATE_BASIS_NET_FLAG,
    SSA_RATE_BASIS_GROSS_FLAG,
    SSA_RATE_BASIS_ACC,
    CASH_DISC_TERMS,
    CASH_DISC_TERMS_DAYS,
    CASH_DISC_TERMS_NET,
    ORDER_SEQUENCE,
    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    CS_VENDOR_ID,
    load_building,
    bic_freight_allow,
    bic_freight_allow_type,
    creation_date,
    bic_freightbill,
    bic_freightbill_type,
    creation_date,
    bic_backhaul,
    bic_backhaul_type,
    creation_date,
    bic_flag_prepaid,
    bic_flag_prepay_and_add,
    bic_flag_freight_bill,
    bic_flag_backhaul,
    bic_external_backhaul,
    bic_ssa_allow_percent,
    bic_ssa_allow_basis,
    bic_ssa_flag,
    bic_ssa_flag,
    bic_ssa_allow_account,
    bic_terms_percent,
    bic_terms_days,
    bic_terms_net_days,
    bic_order_sequence,
    request_id,
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM VendorCostingInfo
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    LOAD_BUILDING = EXCLUDED.LOAD_BUILDING,
    FREIGHT_ALLOW_BUY_RATE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE,
    FREIGHT_ALLOW_BUY_RATE_TYPE = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_TYPE,
    FREIGHT_ALLOW_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_ALLOW_BUY_RATE_CHANGED,
    FREIGHT_BILL_BUY_RATE = EXCLUDED.FREIGHT_BILL_BUY_RATE,
    FREIGHT_BILL_BUY_RATE_TYPE = EXCLUDED.FREIGHT_BILL_BUY_RATE_TYPE,
    FREIGHT_BILL_BUY_RATE_CHANGED = EXCLUDED.FREIGHT_BILL_BUY_RATE_CHANGED,
    BACKHAUL_BUY_RATE = EXCLUDED.BACKHAUL_BUY_RATE,
    BACKHAUL_BUY_RATE_TYPE = EXCLUDED.BACKHAUL_BUY_RATE_TYPE,
    BACKHAUL_BUY_RATE_CHANGED = EXCLUDED.BACKHAUL_BUY_RATE_CHANGED,
    PRE_PAY_FLAG = EXCLUDED.PRE_PAY_FLAG,
    PRE_PAY_ADD_FLAG = EXCLUDED.PRE_PAY_ADD_FLAG,
    FREIGHT_BILL_FLAG = EXCLUDED.FREIGHT_BILL_FLAG,
    BACKHAUL_FLAG = EXCLUDED.BACKHAUL_FLAG,
    EXTERNAL_BACKHAUL_FLAG = EXCLUDED.EXTERNAL_BACKHAUL_FLAG,
    SSA_RATE = EXCLUDED.SSA_RATE,
    SSA_RATE_TYPE = EXCLUDED.SSA_RATE_TYPE,
    SSA_RATE_BASIS_NET_FLAG = EXCLUDED.SSA_RATE_BASIS_NET_FLAG,
    SSA_RATE_BASIS_GROSS_FLAG = EXCLUDED.SSA_RATE_BASIS_GROSS_FLAG,
    SSA_RATE_BASIS_ACC = EXCLUDED.SSA_RATE_BASIS_ACC,
    CASH_DISC_TERMS = EXCLUDED.CASH_DISC_TERMS,
    CASH_DISC_TERMS_DAYS = EXCLUDED.CASH_DISC_TERMS_DAYS,
    CASH_DISC_TERMS_NET = EXCLUDED.CASH_DISC_TERMS_NET,
    ORDER_SEQUENCE = EXCLUDED.ORDER_SEQUENCE,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY;
    
'''

post_cur.execute(vci_query)
vci_count = post_cur.rowcount
vci_count

799204

In [16]:
edi_query = f'''

WITH CloneRecords AS (
SELECT 
        af.facility_code::numeric AS facility_code,
        stg.*  
        
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, EDIData AS (
    SELECT 
        vm.cs_vendor_id,
        temp.*
    FROM CloneRecords temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

-- Insert or upsert into XXPO_VENDOR_EDI_INFO
INSERT INTO XXPO_VENDOR_EDI_INFO (
    CS_VENDOR_ID,
    VENDOR_CONTINUITY_REPLEN_FLG,
    DUNS_REMITT_NO,
    DUNS_NO,
    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    cs_vendor_id,
    vendor_continuity_replen_flg,
    bic_duns_remit_no,
    bic_duns_no,
    request_id,
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM EDIData
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
    DUNS_REMITT_NO = EXCLUDED.DUNS_REMITT_NO,
    DUNS_NO = EXCLUDED.DUNS_NO,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY


'''
post_cur.execute(edi_query)
edi_count = post_cur.rowcount
edi_count

799204

In [18]:
vsp_query = f'''
WITH CloneRecords AS (

SELECT 
        af.facility_code::numeric AS facility_code,
        stg.*  
    FROM XXPO_ORCL2CSPOMS_VEND_STG stg
    CROSS JOIN (
        SELECT facility_code
        FROM cspoms_facility_master
        WHERE inactive_date IS NULL
    ) af
    WHERE stg.process_status IN ('N')
    AND stg.Request_id = '593'
    AND (
        stg.vendor_foodlink_flg IN ('1', '2') 
        OR stg.vendor_freight_flg = 'Y'
        OR stg.bic_vendor_facility IN (14, 21)
    )
    AND stg.bic_vendor_facility != af.facility_code::numeric

)

-- Step 2: Join the cross-joined result with XXPO_VENDOR_MASTER to get cs_vendor_id
, VendorShipPoint AS (
    SELECT 
        vm.cs_vendor_id,
        temp.*
    FROM CloneRecords temp
    JOIN XXPO_VENDOR_MASTER vm
        ON temp.bic_vendor_number = vm.vendor_number
        AND LPAD(temp.facility_code::VARCHAR, 3, '0') = vm.vendor_facility 
)

INSERT INTO XXPO_VENDOR_SHIP_POINT (
    CS_VENDOR_ID,
    VENDOR_NUMBER,
    VENDOR_FACILITY,
    PROCESS_STATUS,
    REQUEST_ID,
    CREATION_DATE,
    LAST_UPDATE_DATE,
    CREATED_BY,
    LAST_UPDATED_BY
)
SELECT
    CS_VENDOR_ID,
    bic_vendor_number,
    LPAD(facility_code::VARCHAR, 3, '0'),
    process_status,
    -999,
    current_timestamp,
    current_timestamp,
    'CSPOMS',
    'CSPOMS'
FROM VendorShipPoint
ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
    VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
    REQUEST_ID = EXCLUDED.REQUEST_ID,
    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY;
    
'''
post_cur.execute(vsp_query)
vsp_count = post_cur.rowcount
vsp_count

799204